# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [ ]:
import pandas as pd
import numpy as np
import glob
pd.options.mode.chained_assignment = None

DATA_FOLDER = './Data/' # Use the data folder provided in Tutorial 02 - Intro to Pandas.

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average per month* of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

In [ ]:
countries = "guinea" #countries = ["guinea", "liberia", "sl"]

path = DATA_FOLDER+"ebola/"+countries+"_data/"

files = glob.glob(path+"*.csv")
cols = ["Date","Description","Totals"]

df = pd.concat((pd.read_csv(f,usecols=cols,parse_dates=['Date']) for f in files),ignore_index=True)

In [ ]:
df.index = df.Date

In [ ]:
new_cases = df[df.Description=="Total new cases registered so far"].Totals.astype(str).astype(int)
deaths = df[df.Description=="Total deaths (confirmed + probables + suspects)"].Totals.astype(str).astype(int)

In [ ]:
avg_new_cases = new_cases.resample('M').mean()
avg_deaths = deaths.resample('M').mean()

In [ ]:
pd.DataFrame(avg_new_cases.values,avg_new_cases.index.month)
df_deaths = pd.DataFrame({"avg deaths":avg_deaths.values,"Month":avg_deaths.index.month})
df_deaths
#avg_deaths.index.month

## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

In [ ]:
path=DATA_FOLDER+"microbiome/"
MID_files = glob.glob(path+"MID*.xls")

After we extract all the 9 spreadsheets from the folder, the metadata file is imported into a DataFrame and we can see that the file gives two informations (Group and Sample) for each MID file.

In [ ]:
df_metadata=pd.read_excel('Data/microbiome/metadata.xls',index_col='BARCODE')
df_metadata

Each spreadsheet is composed of two columns : the name of the bacteria and the number found in the patient.
We define the name of the bacteria as index for each file and use the name of the file (MID1, MID2, ...) as column name in order to anticipate the merge with the metadata file. 
The 9 spreadsheets are imported into a single DataFrame through a column-wise concatenation (axis=1) with respect to the files indices.  
The goal is to add the information contained in the metadata file as columns in our DataFrame that's why we need to transpose our DataFrame in order to have the MID files in rows and the bacteria counts + metadata informations in columns.

In [ ]:
df_microbiome=pd.concat((pd.read_excel(f,index_col=0,header=None,names=[f[f.find('MID'):f.find('MID')+4]]) for f in MID_files),axis=1)
df_microbiome=df_microbiome.transpose()
df_microbiome.index.names=['BARCODE']
df_microbiome


Then we can add to our DataFrame the information contained in the metadata file, doing an inner join which is the default method for a merge in *Pandas*. The only column which is the same between the two DataFrame is the index one (BARCODE) and this needs to be specify in the merge function with the left_index and right_index arguments.

In [ ]:
df_merged=pd.merge(df_microbiome, df_metadata, left_index=True, right_index=True)
df_merged

Finally, we can check the required constraints which are :
- the Nan values needs to be replaced by 'unknown'
- the index needs to be unique, which is the case because we use the name of each spreadsheet as indices

In [ ]:
df_merged.fillna('unknown',inplace=True)
df_merged

In [ ]:
df_merged.index.is_unique

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
# Write your answer here